In [ ]:
"""
X Post Generator Workflow - An Iterative Tweet Generation System

This notebook demonstrates an iterative LangGraph workflow that:
1. Generates original, humorous tweets on a given topic
2. Evaluates tweets based on humor, originality, virality, and format
3. Optimizes tweets through feedback loops until approved or max iterations reached
4. Maintains history of all tweets and feedback for analysis

The workflow uses three specialized LLMs:
- Generator: Creates funny, viral-worthy tweets
- Evaluator: Critiques tweets against quality criteria
- Optimizer: Refines tweets based on feedback

Perfect example of iterative AI workflows with conditional routing and state management.
"""

# Import necessary libraries and load environment variables
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage
from dotenv import load_dotenv
import operator

load_dotenv(dotenv_path='../.env')

True

In [ ]:
# Initialize generator, evaluator, and optimizer LLMs
generator_llm = ChatGroq(model="compound-beta-mini")
evaluator_llm = ChatGroq(model="llama-3.3-70b-versatile")
optimizer_llm = ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct")

In [ ]:
# Define the tweet evaluation output schema using Pydantic
from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel):
    evaluation: Literal['approved', 'needs_improvement'] = Field(..., description='Final')
    feedback : str = Field(..., description='feedback for the tweet')

In [ ]:
# Prepare evaluator for structured TweetEvaluation output
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [ ]:
# Define the processing state schema for tweets
from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel):
    evaluation: Literal['approved', 'needs_improvement'] = Field(..., description='Final')
    feedback : str = Field(..., description='feedback for the tweet')

class TweetState(TypedDict):
    topic : str
    tweet : str
    evaluation : Literal['approved', 'needs_improvement']
    feedback : str
    iteration: int
    max_iteration : int
    
    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [ ]:
# Generate a new tweet based on the topic and return with history
def generate_tweet(state : TweetState):
    messages = [
        SystemMessage(content='You are a funny and clever Twitter/X influencer'),
        HumanMessage(content=f"""
        Write a short, original and hilarious tweet on the topic : {state['topic']}.
        
        Rules :
        - Do NOT use question-answer format.
        - Max 280 characters.
        - Use Observational humor, irony , sarcasm or cultural references.
        - Think in meme logic, punchlines or relatable takes.
        - Use simple, day to day english.
        """)
    ]
    
    response = generator_llm.invoke(messages).content
    return {'tweet' : response, 'tweet_history' : [response]}

In [ ]:
# Evaluate the tweet and record feedback
def evaluate_tweet(state: TweetState):
    messages = [
        SystemMessage(content='You are a ruthless, no laugh-given Twitter critic. You evaluate tweets based on humor, originality , virality and tweet format.'),
        HumanMessage(content=f"""
        Evaluate the follwing tweet:
        Tweet : "{state['tweet']}"
        
        Use the criteria below to evaluate the tweet:
        1. Originality - Is this fresh, or have you seen it a hundred times before?
        2. Humor - Did it genuinely make you smile, laugh or chuckle?
        3. Punchiness - is it short, sharp and scroll-stopping
        4. Virality Potential - Would people retweet or share it?
        5. Format - Is it a well-formed tweet (not a setup punchline joke, not a Q&A joke , and under 280 characters)?
        """)
    ]
    
    response = structured_evaluator_llm.invoke(messages)
    return {'evaluation' : response.evaluation, 'feedback': response.feedback, 'feedback_history' : [response.feedback]}

In [ ]:
# Refine the tweet based on feedback and increment iteration
def optimize_tweet(state: TweetState):
    messages = [
        SystemMessage(content='You punch up tweets for virality and humor based on given feedback'),
        HumanMessage(content=f"""
        Improve the tweet based on this feedback:
        "{state['feedback']}"
        
        Topic : "{state['topic']}"
        Original Tweet: {state['tweet']}
        
        Rewrite it as a short , viral-worthy tweet. Avoid Q&A styles and stay under 280 characters.
        """)
    ]
    
    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1
    return {'tweet' : response, 'iteration': iteration, 'tweet_history' : [response]}

In [ ]:
# Decide whether to stop or loop back based on evaluation and iteration
def route_evaluation(state: TweetState):
    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else :
        return 'needs_improvement'

In [ ]:
# Build the state graph with generate, evaluate, and optimize steps
graph = StateGraph(TweetState)

graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges('evaluate', route_evaluation, {'approved' : END, 'needs_improvement': 'optimize'})
graph.add_edge('optimize', 'evaluate')

workflow = graph.compile()

In [ ]:
# Display the workflow's ASCII diagram
workflow.get_graph().print_ascii()

          +-----------+             
          | __start__ |             
          +-----------+             
                 *                  
                 *                  
                 *                  
           +----------+             
           | generate |             
           +----------+             
                 *                  
                 *                  
                 *                  
           +----------+             
           | evaluate |             
           +----------+             
          ...         **            
         .              **          
       ..                 *         
+---------+           +----------+  
| __end__ |           | optimize |  
+---------+           +----------+  


In [ ]:
# Set up initial state and run the workflow to get final result
initial_state = {
    'topic' : 'Indian railways',
    'iteration' : 1,
    'max_iteration' : 5
}

result = workflow.invoke(initial_state)
result

{'topic': 'Indian railways',
 'tweet': '"Just boarded the Indian Railways express... aka the world\'s longest therapy session. 3 days, 10 delays, and 1 unbearable seatmate later, I\'m basically a life coach #IndianRailways #TrainLife"',
 'evaluation': 'approved',
 'feedback': "This tweet is a great example of a well-crafted tweet that effectively balances originality, humor, and punchiness. The comparison of the Indian Railways express to 'the world's longest therapy session' is a fresh and humorous take on a common experience. The addition of specific details like '3 days, 10 delays, and 1 unbearable seatmate' adds to the comedic effect and makes the tweet more relatable. The use of hashtags #IndianRailways and #TrainLife also increases its virality potential. The tweet is well-formed, under 280 characters, and does not follow a traditional setup-punchline joke format, making it a strong contender for retweets and shares.",
 'iteration': 1,
 'max_iteration': 5,
 'tweet_history': ['"Ju